In [1]:
import phoebe
import numpy as np
import matplotlib.pyplot as plt
import emcee
import sys
%matplotlib inline

In [2]:
b = phoebe.default_binary(contact_binary = True)

In [3]:
#b.add_constraint('semidetached', 'primary')

In [4]:
#b.add_constraint('semidetached', 'secondary')

In [5]:
b['period@orbit'] = 1
#b['sma@orbit'] = 3.2
#b['q@binary'] = 0.44
# b['incl@orbit'] = 83.5
# b['requiv@primary'] = 1.2
# b['requiv@secondary'] = 0.8
#b['teff@primary'] = 5660.
# b['teff@secondary'] = 5500.

In [6]:
#lc = np.loadtxt('data.lc')
v737 = np.loadtxt('v737per.B')
vdata = v737[:,1]
flux = vdata/(-2.5)
flux10 = 10**flux
lc = v737
lc[:,1] = flux10

In [7]:
b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=0.05*np.ones(len(lc)))

<ParameterSet: 78 parameters | contexts: constraint, compute, figure, dataset>

In [8]:
phoebe.interactive_checks_off()
phoebe.interactive_constraints_off()
b.set_value_all('irrad_method', 'none')

In [9]:
b.flip_constraint('compute_phases', 'compute_times')
b['compute_phases@lc@dataset'] = np.linspace(-0.5,0.5,len(lc))

In [10]:
def lnprob(x, adjpars, priors):
    #Check to see that all values are within the allowed limits:
    if not np.all([priors[i][0] < x[i] < priors[i][1] for i in range(len(priors))]):
         return -np.inf

    for i in range(len(adjpars)):
        b[adjpars[i]] = x[i]
    
    # Let's assume that our priors are uniform on the range of the physical parameter combinations.
    # This is already handled in Phoebe, which will throw an error if the system is not physical,
    # therefore it's easy to implement the lnprior as =0 when system checks pass and =-inf if they don't.
    # Here we'll 'package' this in a simple try/except statement:
    
    try:
        b.run_compute(irrad_method='none')

        # sum of squares of the residuals
        fluxes_model = b['fluxes@model'].interp_value(times=lc[:,0])
        lnp = -0.5*np.sum((fluxes_model-b['value@fluxes@dataset'])**2 / b['value@sigmas@dataset']**2) 

    except:
        lnp = -np.inf

    sys.stderr.write("lnp = %e\n" % (lnp))

    return lnp

In [11]:
from scipy.stats import norm
def run(adjpars, priors, nwalkers, niter):
    ndim = len(adjpars)

   

    p0 = np.array([[p[0] + (p[1]-p[0])*np.random.rand() for p in priors] for i in range(nwalkers)])

#     pool = MPIPool()
#     if not pool.is_master():
#         pool.wait()
#         sys.exit(0)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[adjpars, priors])

    pos, prob, state = sampler.run_mcmc(p0, niter)
    
    print(p0)
    
    for i in range(ndim):
        plt.figure()
        y = sampler.flatchain[:,i]
        n, bins, patches = plt.hist(y, 200, density=1, color="b", alpha=0.45)
        plt.title("Dimension {0:d}".format(i))
        
        mu = np.average(y)
        sigma = np.std(y)       
        print ("mu,", "sigma = ", mu, sigma)

        bf = norm.pdf(bins, mu, sigma)
        l = plt.plot(bins, bf, 'k--', linewidth=2.0)

    #plt.show()
    
    return pos

#     pool.close()



In [ ]:
adjpars = ['sma@orbit', 'incl@orbit', 'q@binary', 'teff@primary', 'teff@secondary']
priors = [(2.5,4), (70.0, 88.0), (0, 1), (5400, 5800), (5400., 5800.)]
nwalkers = 64
niters = 1000
state = None

import time

time1 = time.time()
postion = run(adjpars, priors, nwalkers, niters)
time2 = time.time()

lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 310/310 [00:09<00:00, 31.66it/s]
lnp = -1.211597e+03
lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.10it/s]
lnp = -3.210523e+03
lnp = -inf
100%|██████████| 310/310 [00:09<00:00, 31.58it/s]
lnp = -1.163575e+04
100%|██████████| 310/310 [00:10<00:00, 30.67it/s]
lnp = -1.171227e+03
lnp = -inf
100%|██████████| 310/310 [00:09<00:00, 31.03it/s]
lnp = -3.365275e+03
lnp = -inf
lnp = -inf
lnp = -inf
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 31.00it/s]
lnp = -4.086111e+02
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.91it/s]
lnp = -7.541193e+03
100%|██████████| 310/310 [00:09<00:00, 31.03it/s]
lnp = -9.576261e+03
100%|██████████| 310/310 [00:10<00:00, 30.83it/s]
lnp = -2.180599e+02
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.45it/s]
lnp = -4.677203e+03
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.61it/s]
lnp = -6.163490e+03
lnp = -inf
100%|██████████| 310/310 [00:10<00:00, 30.26it/s]
lnp 

In [ ]:
from matplotlib.pyplot import cm 

mod = b
position = postion
times = lc[:,0]
color=cm.rainbow(np.linspace(0,1,nwalkers))

for i,c in zip(range(nwalkers),color):
    
    mod['sma@orbit'] = position[-1-i,0]
    mod['incl@binary@orbit@component'] = position[-1-i,1]
    mod['q@binary'] = position[-1-i,2]
    mod['teff@primary'] = position[-1-i,3]
    mod['teff@secondary'] = position[-1-i,4]
    mod.run_compute(model='run{}'.format(i))


for i,c in zip(range(nwalkers),color):
    model = mod['fluxes@run{}'.format(i)].interp_value(times=times)

    #plt.figure(1)
    plt.plot(times,model,c=c)
plt.plot(times,lc[:,1],"k.")
plt.xlabel("phases")
plt.ylabel("Flux")


In [ ]:
mod['incl@binary@orbit@component']

In [ ]:
mod['q@binary']

In [ ]:
mod['teff@primary']

In [ ]:
mod['teff@secondary']